In [1]:
import os
import requests
from bs4 import BeautifulSoup as bs
import boto3

In [5]:
# Configuração inicial
url = 'https://www.gov.br/inep/pt-br/acesso-a-informacao/dados-abertos/microdados/censo-da-educacao-superior'

s3 = boto3.client('s3')
bucket_name = 'elasticbeanstalk-us-east-2-047710767346'
subfolder_name = 'Case_A3_3'

# verificar se a pasta Case_A3 existe no bucket
try:
    s3.head_object(Bucket=bucket_name, Key=f'{subfolder_name}/')
except:
    # se a pasta não existe, criar
    s3.put_object(Bucket=bucket_name, Key=f'{subfolder_name}/')

# verificar se a pasta "arquivos_zip" dentro de Case_A3 existe no bucket
try:
    s3.head_object(Bucket=bucket_name, Key=f'{subfolder_name}/arquivos_zip/')
    print(f'A pasta {subfolder_name} já existe.')
except:
    # se a pasta não existe, criar
    s3.put_object(Bucket=bucket_name, Key=f'{subfolder_name}/arquivos_zip/')
    print(f'A pasta {subfolder_name} foi criada.')

A pasta Case_A3_3 foi criada.


In [6]:
# raspando a página
html_bruto = requests.get(url, verify=False) 
html_conteudo = html_bruto.text
html_estruturado = bs(html_bruto.content, 'html.parser')

# Configurando o parser e baixando os dados
html_estruturado = bs(html_bruto.content, 'html.parser')

# separando a div desejada - "parent_div"
parent_div = html_estruturado.find('div', {'id': 'parent-fieldname-text'})

# lista para armazenar os links para download
links_download = []

/usr/lib/python3/dist-packages/urllib3/connectionpool.py:1015: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.gov.br'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


In [9]:
# percorrendo os elementos da div para encontrar os links

for year_element in parent_div.find_all('p', {'class': 'callout'}):
    year = year_element.find('strong').text.strip()
    ul_element = year_element.find_next('ul')

    for li_element in ul_element.find_all('li'):
        link = li_element.find('a')['href']
        links_download.append((year, link))

In [10]:
links_download

[('2022',
  'https://download.inep.gov.br/microdados/microdados_censo_da_educacao_superior_2022.zip'),
 ('2021',
  'https://download.inep.gov.br/microdados/microdados_censo_da_educacao_superior_2021.zip'),
 ('2020',
  'https://download.inep.gov.br/microdados/microdados_censo_da_educacao_superior_2020.zip'),
 ('2019',
  'https://download.inep.gov.br/microdados/microdados_censo_da_educacao_superior_2019.zip'),
 ('2018',
  'https://download.inep.gov.br/microdados/microdados_censo_da_educacao_superior_2018.zip'),
 ('2017',
  'https://download.inep.gov.br/microdados/microdados_censo_da_educacao_superior_2017.zip'),
 ('2016',
  'https://download.inep.gov.br/microdados/microdados_censo_da_educacao_superior_2016.zip'),
 ('2015',
  'https://download.inep.gov.br/microdados/microdados_censo_da_educacao_superior_2015.zip'),
 ('2014',
  'https://download.inep.gov.br/microdados/microdados_censo_da_educacao_superior_2014.zip'),
 ('2013',
  'https://download.inep.gov.br/microdados/microdados_censo_da_

In [11]:
# para impedir que a mensagem de erro apareça

from requests.packages.urllib3.exceptions import InsecureRequestWarning
requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

In [12]:
# itera sobre os links para fazer o download e salvar os arquivos
for year, link in links_download:
    try:
        # faz o download e desativa a verificação do certificado SSL
        arquivo_zip = requests.get(link, stream=True, verify=False)
        arquivo_zip.raise_for_status()  # verifica se houve erro no download

        # envia o arquivo para o bucket do S3 dentro da subpasta "arquivos_zip"
        s3.upload_fileobj(
            arquivo_zip.raw,  # fileobj é o objeto do arquivo a ser enviado. raw é o conteúdo do arquivo e indica que o arquivo não foi aberto (camada de abstração)
            bucket_name,  # nome do bucket
            f'{subfolder_name}/arquivos_zip/Microdados_{year}'  # nome do arquivo no bucket (ano_nome_do_arquivo.zip) e a subpasta (arquivos_zip)
        )

        print(f'O arquivo Microdados_{year} foi enviado para o bucket na subpasta {subfolder_name}/arquivos_zip.')

    except Exception as e:
        print(f'Erro ao baixar o arquivo Microdados_{year}: {e}')

O arquivo Case_A3_3/arquivos_zip/Microdados_2022 foi enviado para o bucket na subpasta Case_A3_3/arquivos_zip.
O arquivo Case_A3_3/arquivos_zip/Microdados_2021 foi enviado para o bucket na subpasta Case_A3_3/arquivos_zip.
O arquivo Case_A3_3/arquivos_zip/Microdados_2020 foi enviado para o bucket na subpasta Case_A3_3/arquivos_zip.
O arquivo Case_A3_3/arquivos_zip/Microdados_2019 foi enviado para o bucket na subpasta Case_A3_3/arquivos_zip.
O arquivo Case_A3_3/arquivos_zip/Microdados_2018 foi enviado para o bucket na subpasta Case_A3_3/arquivos_zip.
O arquivo Case_A3_3/arquivos_zip/Microdados_2017 foi enviado para o bucket na subpasta Case_A3_3/arquivos_zip.
O arquivo Case_A3_3/arquivos_zip/Microdados_2016 foi enviado para o bucket na subpasta Case_A3_3/arquivos_zip.
O arquivo Case_A3_3/arquivos_zip/Microdados_2015 foi enviado para o bucket na subpasta Case_A3_3/arquivos_zip.
O arquivo Case_A3_3/arquivos_zip/Microdados_2014 foi enviado para o bucket na subpasta Case_A3_3/arquivos_zip.
O